<a href="https://colab.research.google.com/github/Myrsa016/Disease-Prediction-from-Symptoms/blob/master/tugasbesarDeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow pandas numpy

In [10]:
# Import library yang diperlukan
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Attention
from tensorflow.keras.models import Model
import re
import unicodedata

# Function untuk membersihkan teks
def preprocess_text(text):
    text = ''.join([c for c in unicodedata.normalize('NFD', text)
                    if unicodedata.category(c) != 'Mn'])
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower().strip()
    return text

# Persiapkan data (contoh menggunakan English-Indonesia)
# Dalam kasus nyata, Anda perlu menggantinya dengan dataset Anda sendiri
en_texts = [
    "hello how are you",
    "good morning",
    "thank you very much",
    "see you later"
]

id_texts = [
    "halo apa kabar",
    "selamat pagi",
    "terima kasih banyak",
    "sampai jumpa lagi"
]

# Preprocessing data
en_texts = [preprocess_text(text) for text in en_texts]
id_texts = [preprocess_text(text) for text in id_texts]

# Tambahkan token khusus
en_texts = ['<start> ' + text + ' <end>' for text in en_texts]
id_texts = ['<start> ' + text + ' <end>' for text in id_texts]

# Tokenisasi
en_tokenizer = Tokenizer(filters='')
en_tokenizer.fit_on_texts(en_texts)
en_sequences = en_tokenizer.texts_to_sequences(en_texts)

id_tokenizer = Tokenizer(filters='')
id_tokenizer.fit_on_texts(id_texts)
id_sequences = id_tokenizer.texts_to_sequences(id_texts)

# Padding sequences
max_len = 20
en_padded = pad_sequences(en_sequences, maxlen=max_len, padding='post')
id_padded = pad_sequences(id_sequences, maxlen=max_len, padding='post')

# Parameter model
vocab_size_en = len(en_tokenizer.word_index) + 1
vocab_size_id = len(id_tokenizer.word_index) + 1
embedding_dim = 256
units = 512
batch_size = 64

# Encoder
encoder_inputs = Input(shape=(max_len,))
enc_emb = Embedding(vocab_size_en, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(units, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_len,))
dec_emb_layer = Embedding(vocab_size_id, embedding_dim)
dec_emb = dec_emb_layer(decoder_inputs)

# Attention layer
attention = Attention()
context_vector = attention([encoder_outputs, dec_emb])

# Menggabungkan context vector dengan embedded input
decoder_combined_context = Concatenate(axis=-1)([context_vector, dec_emb]))

# Decoder LSTM
decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_combined_context, initial_state=encoder_states)

# Dense layer
decoder_dense = Dense(vocab_size_id, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training
model.fit(
    [en_padded, id_padded[:, :-1]],
    id_padded[:, 1:],
    batch_size=batch_size,
    epochs=100,
    validation_split=0.2
)

# Function untuk menerjemahkan teks
def translate_text(text):
    # Preprocessing
    text = preprocess_text(text)
    text = '<start> ' + text + ' <end>'

    # Tokenisasi
    text_seq = en_tokenizer.texts_to_sequences([text])
    text_padded = pad_sequences(text_seq, maxlen=max_len, padding='post')

    # Prediksi
    predicted_seq = model.predict([text_padded, np.zeros((1, max_len))])
    predicted_seq = np.argmax(predicted_seq[0], axis=1)

    # Konversi kembali ke teks
    predicted_words = []
    for idx in predicted_seq:
        if idx > 0:
            word = list(id_tokenizer.word_index.keys())[list(id_tokenizer.word_index.values()).index(idx)]
            if word == '<end>':
                break
            if word != '<start>':
                predicted_words.append(word)

    return ' '.join(predicted_words)

# Contoh penggunaan
test_text = "good morning"
translated = translate_text(test_text)
print(f"Input: {test_text}")
print(f"Terjemahan: {translated}")

SyntaxError: unmatched ')' (<ipython-input-10-ceb9c153ff9b>, line 82)

In [9]:
# Import library yang diperlukan
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Attention, Concatenate # Import Concatenate
from tensorflow.keras.models import Model
import re
import unicodedata

# ... (rest of your code remains the same) ...

# Decoder
# ... (other decoder parts) ...

# Attention layer
attention = Attention()
context_vector = attention([encoder_outputs, dec_emb])

# Menggabungkan context vector dengan embedded input using Concatenate layer
# decoder_combined_context = tf.concat([context_vector, dec_emb], axis=-1) # Replaced with the line below
decoder_combined_context = Concatenate(axis=-1)([context_vector, dec_emb])

# Decoder LSTM
# ... (rest of your code remains the same) ...